In [7]:
import numpy as np

# Carga el archivo .npz
archive_title_npz = np.load('movie_titles.npz')
archive_movie_npz= np.load('movie_recommend.npz')

# Claves disponibles en el archivo de títulos
print("Claves disponibles en el archivo de títulos:", archive_title_npz.files)

# claves disponibles en el archivo de recomendaciones de películas
print("Claves disponibles en el archivo de recomendaciones de películas:", archive_movie_npz.files)


# Obtiene el array de títulos
title_array = archive_title_npz['titles']
print("Forma del array de títulos:", title_array.shape)
print("Tipo de datos del array de títulos:", title_array.dtype)
print("Tamaño total del array de títulos:", title_array.size)

# Obtiene el array de recomendaciones de películas
movie_array = archive_movie_npz['data']
print("Forma del array de recomendaciones de películas:", movie_array.shape)
print("Tipo de datos del array de recomendaciones de películas:", movie_array.dtype)
print("Tamaño total del array de recomendaciones de películas:", movie_array.size)

# Obtiene el formato, la forma y la información de los índices
format_array = archive_movie_npz['format']
print("Formato del array de recomendaciones de películas:", format_array)

shape_array = archive_movie_npz['shape']
print("Forma del array de recomendaciones de películas:", shape_array)

row_array = archive_movie_npz['row']
print("Forma del array de índices de fila:", row_array.shape)
print("Tipo de datos del array de índices de fila:", row_array.dtype)
print("Tamaño total del array de índices de fila:", row_array.size)

col_array = archive_movie_npz['col']
print("Forma del array de índices de columna:", col_array.shape)
print("Tipo de datos del array de índices de columna:", col_array.dtype)
print("Tamaño total del array de índices de columna:", col_array.size)



Claves disponibles en el archivo de títulos: ['titles']
Claves disponibles en el archivo de recomendaciones de películas: ['format', 'shape', 'data', 'row', 'col']
Forma del array de títulos: (3706,)
Tipo de datos del array de títulos: <U75
Tamaño total del array de títulos: 3706
Forma del array de recomendaciones de películas: (1000209,)
Tipo de datos del array de recomendaciones de películas: int8
Tamaño total del array de recomendaciones de películas: 1000209
Formato del array de recomendaciones de películas: coo
Forma del array de recomendaciones de películas: [6040 3706]
Forma del array de índices de fila: (1000209,)
Tipo de datos del array de índices de fila: int32
Tamaño total del array de índices de fila: 1000209
Forma del array de índices de columna: (1000209,)
Tipo de datos del array de índices de columna: int32
Tamaño total del array de índices de columna: 1000209


In [11]:
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds

# Cargar los datos de los archivos .npz
archive_title_npz = np.load('movie_titles.npz')
archive_movie_npz = np.load('movie_recommend.npz')

# Obtener la matriz de recomendaciones de películas
row = archive_movie_npz['row']
col = archive_movie_npz['col']
data = archive_movie_npz['data']
shape = archive_movie_npz['shape']
ratings_matrix = coo_matrix((data, (row, col)), shape=shape).astype(float)

# Aplicar la descomposición de valores singulares
U, sigma, Vt = svds(ratings_matrix, k=50)

# Construir la matriz truncada
sigma = np.diag(sigma)
ratings_truncated = np.dot(np.dot(U, sigma), Vt)

# Obtener la lista de títulos de películas
titles = archive_title_npz['titles']

def recomendar_peliculas(user_ratings, num_recomendaciones=5):
    # Calcular las puntuaciones de las películas no vistas
    scores = np.dot(ratings_truncated[user_ratings], Vt.T)
    # Obtener las posiciones de las películas ordenadas por puntuación
    ranked_movies = np.argsort(-scores, axis=1)
    # Obtener las mejores num_recomendaciones películas para cada usuario
    recommendations = [titles[ranked_movies[i, :num_recomendaciones]] for i in range(len(user_ratings))]
    return recommendations


user_ratings = [0, 10, 20]
recomendaciones = recomendar_peliculas(user_ratings)
print(recomendaciones)



[array(['Toy Story (1995)', 'Hunchback of Notre Dame, The (1996)',
       'Star Wars: Episode IV - A New Hope (1977)', 'Titanic (1997)',
       'Christmas Story, A (1983)'], dtype='<U75'), array(['To Kill a Mockingbird (1962)', 'Antz (1998)',
       "Ferris Bueller's Day Off (1986)", 'Ben-Hur (1959)',
       'Toy Story (1995)'], dtype='<U75'), array(['Snow White and the Seven Dwarfs (1937)', 'Apollo 13 (1995)',
       'Saving Private Ryan (1998)', 'Aladdin (1992)',
       'Run Lola Run (Lola rennt) (1998)'], dtype='<U75')]
